<a href="https://colab.research.google.com/github/mlaricobar/datathon-interbank-2019-solution/blob/master/%5BDefinitive%20Solution%5D%20Model%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADD XGBOOST AND RANDOM FOREST

### Instalación de Librerías necesarias

In [1]:
!pip install catboost==0.17.5

     |████████████████████████████████| 62.7MB 136kB/s 



### Lectura de Datos

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competiciones/'[03] Datathon Interbank'/

/gdrive/My Drive/Competiciones/[03] Datathon Interbank


In [4]:
!ls -lh '[01] Intermediate'

total 2.5G
-rw------- 1 root root  33M Nov 27 23:19 df_probs_test_renta_3.csv
-rw------- 1 root root  32M Nov 27 23:18 df_probs_test_renta_4.csv
-rw------- 1 root root  16M Nov 27 23:19 df_probs_train_renta_3.csv
-rw------- 1 root root  14M Nov 27 23:18 df_probs_train_renta_4_final.csv
-rw------- 1 root root  25M Nov 19 18:38 ds_base_all.csv
-rw------- 1 root root 524M Nov 14 21:29 ds_campana_feat.csv
-rw------- 1 root root  43M Nov 19 18:31 ds_digital.csv
-rw------- 1 root root 410M Nov 14 21:28 ds_rcc_feat.csv
-rw------- 1 root root 1.4G Nov 21 11:09 ds_rcc_feat_v2.csv
-rw------- 1 root root 7.1M Nov 19 18:19 ds_reniec.csv
-rw------- 1 root root 9.3M Nov 19 17:58 ds_sunat.csv
-rw------- 1 root root  13M Nov 19 18:18 ds_vehicular.csv


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr

from sklearn import preprocessing
from sklearn.impute import SimpleImputer

In [0]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [0]:
df_base_all = optimizar_df(pd.read_csv('[01] Intermediate/ds_base_all.csv'))
df_vehicular_ds = optimizar_df(pd.read_csv('[01] Intermediate/ds_vehicular.csv'))
df_reniec_ds = optimizar_df(pd.read_csv('[01] Intermediate/ds_reniec.csv'))
df_sunat_ds = optimizar_df(pd.read_csv('[01] Intermediate/ds_sunat.csv'))
df_digital_ds = optimizar_df(pd.read_csv('[01] Intermediate/ds_digital.csv'))
df_campana_ds = optimizar_df(pd.read_csv("[01] Intermediate/ds_campana_feat.csv"))
df_rcc_ds = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_feat_v2.csv"))

In [0]:
df_base_all["ratio_1"] = df_base_all["linea_ofrecida"] / df_base_all["ingreso_neto"]
df_base_all["ratio_2"] = df_base_all["cem"] / df_base_all["ingreso_neto"]
df_base_all["ratio_3"] = df_base_all["cem"] / df_base_all["linea_ofrecida"]
df_base_all["ratio_4"] = df_base_all["ingreso_neto"] / df_base_all["cem"]
df_base_all["ratio_5"] = df_base_all["linea_ofrecida"] / df_base_all["cem"]

In [9]:
df_base_all.head()

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5
0,165747_201901,165747,201901,0.0,0.0,-5.00000,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.000000
1,65751_201901,65751,201901,1.0,1.0,342.98999,687.0,3083.0,4600,0.0,0.0,1.492053,0.222835,0.149348,4.487628,6.695779
2,58047_201901,58047,201901,0.0,0.0,-5.00000,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.780220
3,113221_201901,113221,201901,0.0,0.0,-5.00000,582.0,2153.0,4300,0.0,0.0,1.997213,0.270320,0.135349,3.699313,7.388316
4,142634_201901,142634,201901,0.0,0.0,-5.00000,4058.0,7342.0,33000,0.0,0.0,4.494688,0.552710,0.122970,1.809266,8.132085


In [0]:
#predictores

### Feature Engineering

#### Consolidar Dataset

In [0]:
dataset = df_base_all\
                .merge(df_reniec_ds, how="left", on=["id_persona"])\
                .merge(df_sunat_ds, how="left", on=["id_persona"])\
                .merge(df_vehicular_ds, how="left", on=["id_persona"])\
                .merge(df_rcc_ds, how="left", on=["id_persona", "codmes"])\
                .merge(df_campana_ds, how="left", on=["id_persona", "codmes"])\
                .merge(df_digital_ds, how="left", on=["id_persona", "codmes"])\
                .reset_index(drop=True)

In [11]:
dataset.head()

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta_sum,Grupo_01,Grupo_02,Grupo_03,Grupo_04,Grupo_05,Grupo_06,Grupo_07,Grupo_08,Grupo_09,Grupo_10,Grupo_11,Grupo_12,Grupo_13,Grupo_14,Grupo_15,Grupo_16,...,time_ctasimple_sum,time_ctasimple_mean,time_ctasimple_std,time_mllp_sum,time_mllp_mean,time_mllp_std,time_mllst_sum,time_mllst_mean,time_mllst_std,time_ctasld_sum,time_ctasld_mean,time_ctasld_std,time_tc_sum,time_tc_mean,time_tc_std,time_total_sum,time_total_mean,time_total_std,busqtc_sum,busqtc_mean,busqvisa_sum,busqvisa_mean,busqamex_sum,busqamex_mean,busqmc_sum,busqmc_mean,busqcsimp_sum,busqcsimp_mean,busqmill_sum,busqmill_mean,busqcsld_sum,busqcsld_mean,num_busq_sum,num_busq_mean,num_busq_std,busq_sum,busq_mean,n_pag_sum,n_pag_mean,n_pag_std
0,165747_201901,165747,201901,0.0,0.0,-5.00000,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.000000,0.773585,4.0,1.0,6.0,5.0,5.0,1.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65751_201901,65751,201901,1.0,1.0,342.98999,687.0,3083.0,4600,0.0,0.0,1.492053,0.222835,0.149348,4.487628,6.695779,0.566038,4.0,2.0,6.0,5.0,8.0,2.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58047_201901,58047,201901,0.0,0.0,-5.00000,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.780220,0.603774,4.0,1.0,6.0,5.0,9.0,9.0,221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,113221_201901,113221,201901,0.0,0.0,-5.00000,582.0,2153.0,4300,0.0,0.0,1.997213,0.270320,0.135349,3.699313,7.388316,0.660377,3.0,2.0,6.0,5.0,2.0,3.0,189.0,0.0,0.0,0.0,0.0,0.0,0.0,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,3.0,1.154701
4,142634_201901,142634,201901,0.0,0.0,-5.00000,4058.0,7342.0,33000,0.0,0.0,4.494688,0.552710,0.122970,1.809266,8.132085,0.735849,4.0,2.0,6.0,5.0,1.0,1.0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
dataset["ratio_6"] = dataset["linea_ofrecida"] / dataset["U10M_mto_saldo_mean"]
dataset["ratio_7"] = dataset["ingreso_neto"] / dataset["U10M_mto_saldo_mean"]
dataset["ratio_8"] = dataset["cem"] / dataset["U10M_mto_saldo_mean"]

In [13]:
for col_ in dataset.columns:
  if dataset.loc[dataset[col_] == np.inf, :].shape[0] > 0:
    print(col_)
    dataset.loc[dataset[col_] == np.inf, col_] = np.nan

ratio_4
ratio_5


In [0]:
target_adqui = "codtarget"
target_class = "new_target"
target_regre = "margen"
avoid_columns = [c for c in dataset.columns if "codmes_rcc_first" in c or "codmes_rcc_last" in c or "codmes_campana_first" in c or "codmes_campana_last" in c]
main_columns = ["prediction_id", "id_persona", "codmes", target_adqui, target_class, target_regre]
predictores = [c for c in dataset.columns if c not in (main_columns + avoid_columns)]

In [0]:
month_list = list(sorted(dataset["codmes"].unique()))

### Preparar Marco de Evaluación

In [16]:
df_base_all.head()

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5
0,165747_201901,165747,201901,0.0,0.0,-5.00000,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.000000
1,65751_201901,65751,201901,1.0,1.0,342.98999,687.0,3083.0,4600,0.0,0.0,1.492053,0.222835,0.149348,4.487628,6.695779
2,58047_201901,58047,201901,0.0,0.0,-5.00000,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.780220
3,113221_201901,113221,201901,0.0,0.0,-5.00000,582.0,2153.0,4300,0.0,0.0,1.997213,0.270320,0.135349,3.699313,7.388316
4,142634_201901,142634,201901,0.0,0.0,-5.00000,4058.0,7342.0,33000,0.0,0.0,4.494688,0.552710,0.122970,1.809266,8.132085


### Siendo sincero :( 
Se encontró un Leakage en el dataset

In [17]:
# Hay una hipótesis sobre aquellos clientes que son gestionados nuevamente
for month_to_analyze in [201901, 201902, 201903, 201904]:
  months_after_list = [c for c in month_list if c > month_to_analyze]
  target_list_ = df_base_all.loc[(df_base_all["codmes"] == month_to_analyze) & (df_base_all["id_persona"].isin(df_base_all.loc[df_base_all["codmes"].isin(months_after_list), "id_persona"])), "new_target"]
  print("MEAN TARGET: {0} IN MONTH {1} AND {2:,} CUSTOMERS".format(round(target_list_.mean(), 2), month_to_analyze, len(target_list_)))

MEAN TARGET: 0.0 IN MONTH 201901 AND 27,074 CUSTOMERS
MEAN TARGET: 0.0 IN MONTH 201902 AND 19,462 CUSTOMERS
MEAN TARGET: 0.0 IN MONTH 201903 AND 26,099 CUSTOMERS
MEAN TARGET: 0.0 IN MONTH 201904 AND 26,271 CUSTOMERS


In [0]:
month_to_analyze = 201907

In [0]:
df_base_tmp_ = df_base_all.loc[(df_base_all["codmes"] == month_to_analyze), ["prediction_id", "id_persona", "codmes", "new_target"]]

In [0]:
df_base_hist_ = df_base_all.loc[(df_base_all["codmes"] < month_to_analyze) & (df_base_all["id_persona"].isin(df_base_tmp_["id_persona"].unique())), :].groupby("id_persona", as_index=False).agg({"prediction_id":"count"}).sort_values(by="prediction_id", ascending=False)

In [42]:
df_base_tmp_.loc[df_base_tmp_["id_persona"].isin(df_base_hist_.loc[df_base_hist_["prediction_id"] == 3, "id_persona"].tolist()), "new_target"].value_counts()

0.0    962
1.0     34
Name: new_target, dtype: int64

In [27]:
df_base_tmp_.head()

,prediction_id
329630,123381_201907
329631,131234_201907
329632,17651_201907
329633,224194_201907
329634,11651_201907


Viendo que todos los clientes que fueron gestionados en un mes determinado y luego son gestionados en el mes siguiente tienen un promedio de target igual a 0, tendrán el valor de 0 en el codtarget y asumiremos que tendrán una rentabilidad de -5.00000.

Para aprovechar el leakage haremos el recorrido sobre los meses de la base de test para identificar esos clientes y por ende etiquetarlos con codtarget = 0, new_target = 0 y margen = -5.000 que serán enviadas en el archivo de submission y también serán utilizados como base de entrenamiento para tener mas registros que permitan discriminar los rentables de esos que no lo son porque no han adquirido el producto ofrecido.

In [25]:
df_base_all.head()

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5
0,165747_201901,165747,201901,0.0,0.0,-5.00000,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.000000
1,65751_201901,65751,201901,1.0,1.0,342.98999,687.0,3083.0,4600,0.0,0.0,1.492053,0.222835,0.149348,4.487628,6.695779
2,58047_201901,58047,201901,0.0,0.0,-5.00000,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.780220
3,113221_201901,113221,201901,0.0,0.0,-5.00000,582.0,2153.0,4300,0.0,0.0,1.997213,0.270320,0.135349,3.699313,7.388316
4,142634_201901,142634,201901,0.0,0.0,-5.00000,4058.0,7342.0,33000,0.0,0.0,4.494688,0.552710,0.122970,1.809266,8.132085


In [0]:
for month_to_analyze in [201901, 201902, 201903, 201904]:
  

In [0]:
list_leakage_id = []
for month_to_analyze in [201905, 201906, 201907]:
  months_after_list = [c for c in month_list if c > month_to_analyze]
  list_leakage_id += df_base_all.loc[(df_base_all["codmes"] == month_to_analyze) &
                           (df_base_all["id_persona"].isin(df_base_all.loc[df_base_all["codmes"].isin(months_after_list), "id_persona"])), "prediction_id"].tolist()
  

In [19]:
len(list_leakage_id)

36069

In [0]:
dataset.loc[dataset["prediction_id"].isin(list_leakage_id), "margen"] = -5

Se han ganado 36K registros adicionales y además 36K con respuestas de target seguras que son las que permiten obtener un buen score pero que no son implementables en la vida real.

Por eso el planteamiento del problema se debió realizar de manera cuidadosa para evitar este tipo de casos.

In [20]:
#df_train = dataset.loc[~(dataset["codmes"].isin([201905, 201906, 201907])) | (dataset["prediction_id"].isin(list_leakage_id)) , :].reset_index(drop=True)
#df_test = dataset.loc[dataset["codmes"].isin([201905, 201906, 201907]) & ~(dataset["prediction_id"].isin(list_leakage_id)), :].reset_index(drop=True)
df_train = dataset.loc[~(dataset["codmes"].isin([201905, 201906, 201907])) , :].reset_index(drop=True)
df_test = dataset.loc[dataset["codmes"].isin([201905, 201906, 201907]), :].reset_index(drop=True)

nrows, nrows_train, nrows_test = dataset.shape[0], df_train.shape[0], df_test.shape[0]

print("Cantidad de filas: {0:,}\n\t- Train:\t{1:,} filas\t({2:}%)\n\t- Test:\t\t{3:,} filas\t({4:}%)".format(nrows,
                                                                                                                                             nrows_train,
                                                                                                                                             round((100 * (nrows_train/nrows)), 2),
                                                                                                                                             nrows_test,
                                                                                                                                             round((100 * (nrows_test/nrows)), 2)
                                                                                                                                             ))

Cantidad de filas: 391,350
	- Train:	212,665 filas	(54.34%)
	- Test:		178,685 filas	(45.66%)


In [21]:
df_train.head()

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta_sum,Grupo_01,Grupo_02,Grupo_03,Grupo_04,Grupo_05,Grupo_06,Grupo_07,Grupo_08,Grupo_09,Grupo_10,Grupo_11,Grupo_12,Grupo_13,Grupo_14,Grupo_15,Grupo_16,...,time_mllp_sum,time_mllp_mean,time_mllp_std,time_mllst_sum,time_mllst_mean,time_mllst_std,time_ctasld_sum,time_ctasld_mean,time_ctasld_std,time_tc_sum,time_tc_mean,time_tc_std,time_total_sum,time_total_mean,time_total_std,busqtc_sum,busqtc_mean,busqvisa_sum,busqvisa_mean,busqamex_sum,busqamex_mean,busqmc_sum,busqmc_mean,busqcsimp_sum,busqcsimp_mean,busqmill_sum,busqmill_mean,busqcsld_sum,busqcsld_mean,num_busq_sum,num_busq_mean,num_busq_std,busq_sum,busq_mean,n_pag_sum,n_pag_mean,n_pag_std,ratio_6,ratio_7,ratio_8
0,165747_201901,165747,201901,0.0,0.0,-5.00000,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.000000,0.773585,4.0,1.0,6.0,5.0,5.0,1.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038430,0.040261,0.000023
1,65751_201901,65751,201901,1.0,1.0,342.98999,687.0,3083.0,4600,0.0,0.0,1.492053,0.222835,0.149348,4.487628,6.695779,0.566038,4.0,2.0,6.0,5.0,8.0,2.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177250,0.118796,0.026472
2,58047_201901,58047,201901,0.0,0.0,-5.00000,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.780220,0.603774,4.0,1.0,6.0,5.0,9.0,9.0,221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122254,0.061603,0.006181
3,113221_201901,113221,201901,0.0,0.0,-5.00000,582.0,2153.0,4300,0.0,0.0,1.997213,0.270320,0.135349,3.699313,7.388316,0.660377,3.0,2.0,6.0,5.0,2.0,3.0,189.0,0.0,0.0,0.0,0.0,0.0,0.0,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,3.0,1.154701,0.150854,0.075532,0.020418
4,142634_201901,142634,201901,0.0,0.0,-5.00000,4058.0,7342.0,33000,0.0,0.0,4.494688,0.552710,0.122970,1.809266,8.132085,0.735849,4.0,2.0,6.0,5.0,1.0,1.0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.182122,0.040519,0.022396


In [0]:
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

from scipy.optimize import differential_evolution

In [0]:
seed_val = 41

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

ctb_regr_params = {'loss_function':'RMSE',
              'eval_metric':'RMSE',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

lgb_params = {
                "objective": "binary",
                "metric": "auc",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

lgb_regr_params = {
                "objective": "regression",
                "metric": "l2",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

xgb_params = {
                'base_score': 0.5,
                'booster': 'gbtree',
                'colsample_bylevel': 1,
                'colsample_bynode': 1,
                'colsample_bytree': 0.8,
                'gamma': 0,
                'learning_rate': 0.1,
                'max_delta_step': 0,
                'max_depth': 5,
                'min_child_weight': 1,
                'missing': None,
                'n_estimators': 1000,
                'nthread': -1,
                'objective': 'binary:logistic',
                'reg_alpha': 0,
                'reg_lambda': 1,
                'scale_pos_weight': 1,
                'seed': 27,
                'subsample': 0.8,
                'verbosity': 1,
                'eval_metric':"auc"
              }

rf_params = {
              'n_estimators':500,
              'random_state':0,
              'n_jobs': -1,
              'max_depth': 12,
              'class_weight': "balanced",
              "min_samples_leaf" : 20,
              "max_features" : 300
             }

In [0]:
k = 5 # n folds

def model_with_cv(df_train, df_test, target_class, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["prediction_id", target_class]].copy().reset_index(drop=True)
  df_probs_test = df_test[["prediction_id"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["prediction_id", target_class]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  # Generar los Folds
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_class])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t CLASIFICACIÓN")
      print("\t\t 1. CATBOOST")
      ## CatBoost
      cbt_model = CatBoostClassifier(**ctb_params)
      cbt_model.fit(df_train_cv[predictores], df_train_cv[target_class], eval_set=(df_val_cv[predictores], df_val_cv[target_class]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_class])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_class])
      lgb_model = lgb.train(lgb_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Clasificación
      probas_train_cbtm = cbt_model.predict_proba(df_train_cv[predictores])[:, 1]
      probas_val_cbtm = cbt_model.predict_proba(df_val_cv[predictores])[:, 1]

      probas_train_lgbm = lgb_model.predict(df_train_cv[predictores])
      probas_val_lgbm = lgb_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Clasificación
      metric_train_cbtm = roc_auc_score(df_train_cv[target_class], probas_train_cbtm)
      metric_val_cbtm = roc_auc_score(df_val_cv[target_class], probas_val_cbtm)

      metric_train_lgbm = roc_auc_score(df_train_cv[target_class], probas_train_lgbm)
      metric_val_lgbm = roc_auc_score(df_val_cv[target_class], probas_val_lgbm)

      if df_val is not None:
        probas_val_out_cbtm = cbt_model.predict_proba(df_val[predictores])[:, 1]
        metric_val_out_cbtm = roc_auc_score(df_val[target_class], probas_val_out_cbtm)

        probas_val_out_lgbm = lgb_model.predict(df_val[predictores])
        metric_val_out_lgbm = roc_auc_score(df_val[target_class], probas_val_out_lgbm)

      else:
        probas_val_out_cbtm = np.zeros((1,1))
        metric_val_out_cbtm = 0

        probas_val_out_lgbm = np.zeros((1,1))
        metric_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "prob_cbtm"] = probas_val_cbtm
      df_probs_train.loc[valcv, "prob_lgbm"] = probas_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)

      
      df_probs_test["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_test[predictores])[:, 1]
      df_probs_test["prob_lgbm_{}".format(i)] = lgb_model.predict(df_test[predictores])

      if df_val is not None:
        df_probs_val["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_val[predictores])[:, 1]
        df_probs_val["prob_lgbm_{}".format(i)] = lgb_model.predict(df_val[predictores])

        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, "auc_val_out_cbtm": metric_val_out_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, "auc_val_out_lgbm": metric_val_out_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4), round(df_val[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4), round(np.mean(probas_val_out_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4), round(np.std(probas_val_out_cbtm),4)))

      else:

        # Almacenar los resultados
        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4)))
        
        
      list_models_cv.append({"cbtm_model": cbt_model, "lgbm_model": lgb_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  ##################################################################
  ############# PARA ETAPA DE OPTIMIZACIÓN
  ##################################################################

  if df_val is not None:
    df_probs_val["prob_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_cbtm" in c]].mean(axis=1)
    df_probs_val["prob_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_lgbm" in c]].mean(axis=1)

  df_probs_test["prob_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_cbtm" in c]].mean(axis=1)
  df_probs_test["prob_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_lgbm" in c]].mean(axis=1)

  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

##### Adquisition Model

In [0]:
results_adqui, list_models_adqui, df_probs_train_adqui, df_probs_val_adqui, df_probs_test_adqui = model_with_cv(df_train, df_test, target_adqui, predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.136028
0:	learn: 0.6738970	test: 0.6724440	best: 0.6724440 (0)	total: 18ms	remaining: 18s
200:	learn: 0.8061127	test: 0.7898562	best: 0.7898562 (200)	total: 3.23s	remaining: 12.8s
400:	learn: 0.8208062	test: 0.7929595	best: 0.7929595 (400)	total: 6.28s	remaining: 9.38s
600:	learn: 0.8325427	test: 0.7932473	best: 0.7933763 (512)	total: 9.4s	remaining: 6.24s
800:	learn: 0.8430612	test: 0.7938000	best: 0.7938000 (800)	total: 12.5s	remaining: 3.1s
999:	learn: 0.8527017	test: 0.7942081	best: 0.7942535 (984)	total: 15.6s	remaining: 0us
bestTest = 0.7942535281
bestIteration = 984
Shrink model to first 985 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.772556	valid_1's auc: 0.767041
[40]	training's auc: 0.78535	valid_1's auc: 0.776143
[60]	training's auc: 0.794884	valid_1's auc: 0.782244
[80]	training's auc: 0.801861	valid_1's auc: 0.78612
[100]	training's auc: 0.

In [0]:
df_probs_test_adqui.shape

(142616, 13)

In [0]:
#results_adqui, list_models_adqui, df_probs_train_adqui, df_probs_val_adqui, df_probs_test_adqui = model_with_cv(df_train, df_test, target_adqui, predictores, None)

In [0]:
# First Version
pd.DataFrame(results_adqui)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.842622,0.794255,0.860582,0.796075,41
1,0.846026,0.792608,0.847212,0.793052,41
2,0.847536,0.790017,0.853736,0.792627,41
3,0.850533,0.797989,0.879172,0.800990,41
4,0.841775,0.796608,0.874608,0.797567,41


In [0]:
# Second Version
pd.DataFrame(results_adqui)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.852062,0.794254,0.860582,0.796075,41
1,0.842768,0.793075,0.847212,0.793052,41
2,0.854110,0.789462,0.853736,0.792627,41
3,0.844817,0.797847,0.879172,0.800990,41
4,0.841775,0.796608,0.874608,0.797567,41


In [0]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models_adqui[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    print(i)
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models_adqui[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")
    df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)

df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(20)

1
2
3
4


,name,importance_fold_0,importance_fold_1,importance_folds_mean,importance_fold_2,importance_fold_3,importance_fold_4
0,U2M_NA_canal_asignado_new_sum,7.133191,11.935358,8.835271,8.595358,8.134986,8.193045
1,U2M_Combos_producto_new_sum,5.290656,10.614899,7.630872,7.941330,7.341432,6.769271
2,U4M_NA_producto_new_sum,3.882341,7.054947,4.859758,3.130109,5.625869,4.549825
3,flg_exist_hist,4.075892,4.208825,4.199203,3.781342,4.342671,4.694321
4,soc_var1,2.724564,2.355824,2.623938,2.620480,2.694756,2.756826
5,U4M_TLV_canal_asignado_new_sum,1.321338,1.697540,1.841364,2.114231,1.994668,2.162726
6,U6M_NA_producto_new_sum,0.000000,1.354337,1.607534,1.757610,2.756206,2.413967
7,U6M_Adquisicion_producto_new_sum,1.559303,1.538566,1.594681,1.478817,1.900788,1.488837
8,U2M_NA_producto_new_sum,3.039546,0.000000,1.471396,2.114243,0.013483,2.293974
9,U6M_TLV_canal_asignado_new_sum,1.282991,0.968428,1.227789,0.957552,1.440063,1.571162


In [0]:
df_predictores_.to_csv("predictores_first_model.csv", index=False)

##### Rentability Model Classification

###### With ALL

In [0]:
results_renta_1, list_models_renta_1, df_probs_train_renta_1, df_probs_val_renta_1, df_probs_test_renta_1 = model_with_cv(df_train, df_test, target_class, predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.136028
0:	learn: 0.6858625	test: 0.6880222	best: 0.6880222 (0)	total: 16.6ms	remaining: 16.6s
200:	learn: 0.8374787	test: 0.8184593	best: 0.8184624 (199)	total: 3.32s	remaining: 13.2s
400:	learn: 0.8508102	test: 0.8213207	best: 0.8213207 (400)	total: 6.57s	remaining: 9.81s
600:	learn: 0.8619752	test: 0.8226088	best: 0.8226350 (599)	total: 9.81s	remaining: 6.51s
800:	learn: 0.8714315	test: 0.8227091	best: 0.8229067 (718)	total: 13.1s	remaining: 3.25s
999:	learn: 0.8806905	test: 0.8230679	best: 0.8231973 (930)	total: 16.3s	remaining: 0us
bestTest = 0.8231973052
bestIteration = 930
Shrink model to first 931 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.803806	valid_1's auc: 0.792256
[40]	training's auc: 0.819632	valid_1's auc: 0.804878
[60]	training's auc: 0.830394	valid_1's auc: 0.811677
[80]	training's auc: 0.838624	valid_1's auc: 0.815289
[100]	training's

In [0]:
pd.DataFrame(results_renta_1)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.877542,0.823197,0.918896,0.826775,41
1,0.868295,0.824447,0.891269,0.825522,41
2,0.871315,0.824872,0.912757,0.827387,41
3,0.877686,0.825712,0.907588,0.827885,41
4,0.877872,0.827369,0.905526,0.829509,41


In [0]:
pd.DataFrame(results_renta_1)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.865454,0.812883,0.894461,0.813908,41
1,0.872681,0.816804,0.889301,0.817849,41
2,0.866489,0.814111,0.886794,0.817558,41
3,0.873617,0.810741,0.905314,0.811592,41
4,0.874568,0.806605,0.876171,0.805082,41


In [0]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models_renta_1[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    print(i)
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models_renta_1[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")
    df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)

df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(20)

1
2
3
4


,name,importance_fold_0,importance_fold_1,importance_folds_mean,importance_fold_2,importance_fold_3,importance_fold_4
0,U2M_NA_canal_asignado_new_sum,10.366241,7.245178,7.621364,7.961507,6.763669,5.205336
1,U2M_Combos_producto_new_sum,10.606719,6.639597,7.346624,7.700303,6.497628,4.668158
2,flg_exist_hist,2.465572,4.332569,3.545435,2.876560,4.247213,3.912625
3,U6M_NA_producto_new_sum,3.984590,3.838866,3.416313,1.630276,2.810540,5.112478
4,soc_var1,2.863204,3.146656,3.003089,2.940574,3.120535,2.935603
5,U2M_sum_mto_saldo_in_new_producto_TARJETAS COM...,1.706623,2.068247,1.748866,1.514407,1.715991,1.728042
6,ratio_7,1.829069,1.549882,1.447793,1.669403,1.006766,1.083722
7,U2M_NA_producto_new_sum,0.000000,0.000000,1.337730,3.195982,3.359500,0.000000
8,U6M_TLV_canal_asignado_new_sum,1.974550,1.301390,1.324950,1.510224,0.908799,0.794538
9,U6M_Adquisicion_producto_new_sum,0.906523,2.219568,1.315665,1.367642,1.270716,0.673814


In [0]:
df_predictores_.to_csv("predictores_rentability_model.csv", index=False)

###### With Just Contacted

In [0]:
df_train.loc[df_train["codtarget"] == 1].groupby("codmes", as_index=False).agg({"new_target": ["count", "sum", "mean"]})

codmes new_target                  
               count     sum      mean
0  201901       6797  4170.0  0.613506
1  201902       6876  4461.0  0.648778
2  201903       8495  6004.0  0.706769
3  201904       8804  6159.0  0.699568

In [0]:
results_renta_2, list_models_renta_2, df_probs_train_renta_2, df_probs_val_renta_2, df_probs_test_renta_2 = model_with_cv(df_train.loc[df_train[target_adqui] == 1].reset_index(drop=True), df_test, target_class, predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.109302
0:	learn: 0.6443666	test: 0.6477043	best: 0.6477043 (0)	total: 81.9ms	remaining: 1m 21s
200:	learn: 0.7708621	test: 0.7251613	best: 0.7257644 (166)	total: 14.1s	remaining: 56.2s
400:	learn: 0.8021103	test: 0.7226514	best: 0.7257644 (166)	total: 28s	remaining: 41.8s
600:	learn: 0.8208703	test: 0.7196231	best: 0.7257644 (166)	total: 41.7s	remaining: 27.7s
800:	learn: 0.8359344	test: 0.7187715	best: 0.7257644 (166)	total: 55.5s	remaining: 13.8s
999:	learn: 0.8458733	test: 0.7175394	best: 0.7257644 (166)	total: 1m 9s	remaining: 0us
bestTest = 0.7257643938
bestIteration = 166
Shrink model to first 167 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.738055	valid_1's auc: 0.711406
[40]	training's auc: 0.758152	valid_1's auc: 0.720116
[60]	training's auc: 0.775474	valid_1's auc: 0.726444
[80]	training's auc: 0.791103	valid_1's auc: 0.729198
[100]	training's 

In [0]:
results_renta_2, list_models_renta_2, df_probs_train_renta_2, df_probs_val_renta_2, df_probs_test_renta_2 = model_with_cv(df_train.loc[df_train[target_adqui] == 1].reset_index(drop=True), df_test, target_class, predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.109302
0:	learn: 0.6443666	test: 0.6477043	best: 0.6477043 (0)	total: 77.3ms	remaining: 1m 17s
200:	learn: 0.7708621	test: 0.7251613	best: 0.7257644 (166)	total: 13.9s	remaining: 55.1s
400:	learn: 0.8021103	test: 0.7226514	best: 0.7257644 (166)	total: 27.5s	remaining: 41.1s
600:	learn: 0.8208703	test: 0.7196231	best: 0.7257644 (166)	total: 41s	remaining: 27.2s
800:	learn: 0.8359344	test: 0.7187715	best: 0.7257644 (166)	total: 54.4s	remaining: 13.5s
999:	learn: 0.8458733	test: 0.7175394	best: 0.7257644 (166)	total: 1m 7s	remaining: 0us
bestTest = 0.7257643938
bestIteration = 166
Shrink model to first 167 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.738055	valid_1's auc: 0.711406
[40]	training's auc: 0.758152	valid_1's auc: 0.720116
[60]	training's auc: 0.775474	valid_1's auc: 0.726444
[80]	training's auc: 0.791103	valid_1's auc: 0.729198
[100]	training's 

In [0]:
pd.DataFrame(results_renta_2)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.762690,0.725765,0.808588,0.729313,41
1,0.798604,0.710996,0.796474,0.712075,41
2,0.772740,0.710623,0.819479,0.710245,41
3,0.797334,0.709009,0.816881,0.710630,41
4,0.773887,0.714193,0.845807,0.718085,41


In [0]:
pd.DataFrame(results_renta_2)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.762690,0.725765,0.808588,0.729313,41
1,0.798604,0.710996,0.796474,0.712075,41
2,0.772740,0.710623,0.819479,0.710245,41
3,0.797334,0.709009,0.816881,0.710630,41
4,0.773887,0.714193,0.845807,0.718085,41


In [0]:
#predictores

In [0]:
df_probs_train_renta_2_complement = df_train.loc[df_train[target_adqui] != 1, :].copy().reset_index(drop=True)

In [0]:
df_probs_train_renta_2_complement.head(2)

,prediction_id,id_persona,codmes,codtarget,new_target,margen,cem,ingreso_neto,linea_ofrecida,flg_exist_prev_month,flg_exist_hist,ratio_1,ratio_2,ratio_3,ratio_4,ratio_5,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta_sum,Grupo_01,Grupo_02,Grupo_03,Grupo_04,Grupo_05,Grupo_06,Grupo_07,Grupo_08,Grupo_09,Grupo_10,Grupo_11,Grupo_12,Grupo_13,Grupo_14,Grupo_15,Grupo_16,...,time_mllp_sum,time_mllp_mean,time_mllp_std,time_mllst_sum,time_mllst_mean,time_mllst_std,time_ctasld_sum,time_ctasld_mean,time_ctasld_std,time_tc_sum,time_tc_mean,time_tc_std,time_total_sum,time_total_mean,time_total_std,busqtc_sum,busqtc_mean,busqvisa_sum,busqvisa_mean,busqamex_sum,busqamex_mean,busqmc_sum,busqmc_mean,busqcsimp_sum,busqcsimp_mean,busqmill_sum,busqmill_mean,busqcsld_sum,busqcsld_mean,num_busq_sum,num_busq_mean,num_busq_std,busq_sum,busq_mean,n_pag_sum,n_pag_mean,n_pag_std,ratio_6,ratio_7,ratio_8
0,165747_201901,165747,201901,0.0,0.0,-5.0,1.0,1781.0,1700,0.0,0.0,0.954520,0.000561,0.000588,1781.000000,1700.00000,0.773585,4.0,1.0,6.0,5.0,5.0,1.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038430,0.040261,0.000023
1,58047_201901,58047,201901,0.0,0.0,-5.0,182.0,1814.0,3600,0.0,0.0,1.984564,0.100331,0.050556,9.967033,19.78022,0.603774,4.0,1.0,6.0,5.0,9.0,9.0,221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,221.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122254,0.061603,0.006181


In [0]:
df_probs_train_renta_2.head(2)

,prediction_id,new_target,prob_cbtm,prob_lgbm,FOLD
0,65751_201901,1.0,0.715803,0.764294,FOLD_3
1,65634_201901,0.0,0.328104,0.352521,FOLD_5


In [0]:
print(df_probs_train_renta_2.columns.tolist())

['prediction_id', 'new_target', 'prob_cbtm', 'prob_lgbm', 'FOLD']


In [0]:
for i, pair_m in enumerate(list_models_renta_2):
  print(i)
  df_probs_train_renta_2_complement["prob_cbtm_{}".format(i)] = pair_m["cbtm_model"].predict_proba(df_probs_train_renta_2_complement[predictores])[:, 1]
  df_probs_train_renta_2_complement["prob_lgbm_{}".format(i)] = pair_m["lgbm_model"].predict(df_probs_train_renta_2_complement[predictores])

0
1
2
3
4


In [0]:
df_probs_train_renta_2_complement["prob_cbtm"] = df_probs_train_renta_2_complement[[c for c in df_probs_train_renta_2_complement.columns if "prob_cbtm" in c]].mean(axis=1)
df_probs_train_renta_2_complement["prob_lgbm"] = df_probs_train_renta_2_complement[[c for c in df_probs_train_renta_2_complement.columns if "prob_lgbm" in c]].mean(axis=1)

In [0]:
df_probs_train_renta_2_complement = df_probs_train_renta_2_complement[['prediction_id', 'new_target', 'prob_cbtm', 'prob_lgbm']]

In [0]:
df_probs_train_renta_2_complement.shape

(217762, 4)

In [0]:
df_probs_train_renta_2_final = pd.concat([df_probs_train_renta_2, df_probs_train_renta_2_complement], sort=False).reset_index(drop=True)

In [0]:
df_probs_train_renta_2_final.head()

,prediction_id,new_target,prob_cbtm,prob_lgbm,FOLD
0,65751_201901,1.0,0.715803,0.764294,FOLD_3
1,65634_201901,0.0,0.328104,0.352521,FOLD_5
2,86617_201901,1.0,0.785928,0.791304,FOLD_4
3,116521_201901,1.0,0.544718,0.632826,FOLD_4
4,178404_201901,0.0,0.311350,0.302059,FOLD_3


In [0]:
df_probs_train_renta_2_final.shape

(248734, 5)

In [0]:
df_probs_train_renta_2_final.shape

(248734, 5)

In [0]:
pd.DataFrame(results_renta_2)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.762690,0.725765,0.808588,0.729313,41
1,0.798604,0.710996,0.796474,0.712075,41
2,0.772740,0.710623,0.819479,0.710245,41
3,0.797334,0.709009,0.816881,0.710630,41
4,0.773887,0.714193,0.845807,0.718085,41


In [0]:
pd.DataFrame(results_renta_2)

##### Rentability Model Regression

In [0]:
k = 5 # n folds

def renta_regre_model_with_cv(df_train, df_test, target_regres, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["prediction_id", target_regres]].copy().reset_index(drop=True)
  df_probs_test = df_test[["prediction_id"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["prediction_id", target_regres]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  
  # Generar los Folds
  skf = KFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_regres])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t REGRESIÓN")
      print("\t\t 1. CATBOOST")
      cbt_reg_model = CatBoostRegressor(**ctb_regr_params)
      cbt_reg_model.fit(df_train_cv[predictores], df_train_cv[target_regres], eval_set=(df_val_cv[predictores], df_val_cv[target_regres]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_regres])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_regres])
      lgb_reg_model = lgb.train(lgb_regr_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Regresión
      pred_train_cbtm = cbt_reg_model.predict(df_train_cv[predictores])
      pred_val_cbtm = cbt_reg_model.predict(df_val_cv[predictores])

      pred_train_lgbm = lgb_reg_model.predict(df_train_cv[predictores])
      pred_val_lgbm = lgb_reg_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Regresión
      metric_reg_train_cbtm = np.sqrt(mean_squared_error(df_train_cv[target_regre], pred_train_cbtm))
      metric_reg_val_cbtm = np.sqrt(mean_squared_error(df_val_cv[target_regre], pred_val_cbtm))

      metric_reg_train_lgbm = np.sqrt(mean_squared_error(df_train_cv[target_regre], pred_train_lgbm))
      metric_reg_val_lgbm = np.sqrt(mean_squared_error(df_val_cv[target_regre], pred_val_lgbm))

      if df_val is not None:
        pred_val_out_cbtm = cbt_reg_model.predict(df_val[predictores])
        metric_reg_val_out_cbtm = np.sqrt(mean_squared_error(df_val[target_regre], pred_val_out_cbtm))

        pred_val_out_lgbm = lgb_reg_model.predict(df_val[predictores])
        metric_reg_val_out_lgbm = np.sqrt(mean_squared_error(df_val[target_regre], pred_val_out_lgbm))

      else:
        pred_val_out_cbtm = np.zeros((1,1))
        metric_reg_val_out_cbtm = 0

        pred_val_out_lgbm = np.zeros((1,1))
        metric_reg_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "pred_cbtm"] = pred_val_cbtm
      df_probs_train.loc[valcv, "pred_lgbm"] = pred_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)
      
      
      df_probs_test["pred_cbtm_{}".format(i)] = cbt_reg_model.predict(df_test[predictores])
      df_probs_test["pred_lgbm_{}".format(i)] = lgb_reg_model.predict(df_test[predictores])
      
      if df_val is not None:
        df_probs_val["pred_cbtm_{}".format(i)] = cbt_reg_model.predict(df_val[predictores])
        df_probs_val["pred_lgbm_{}".format(i)] = lgb_reg_model.predict(df_val[predictores])

        # Almacenar los resultados
        results_models.append({"rmse_train_cbtm": metric_reg_train_cbtm, "rmse_val_cbtm": metric_reg_val_cbtm, ## CATBOOST
                            "rmse_train_lgbm": metric_reg_train_lgbm, "rmse_val_lgbm": metric_reg_val_lgbm, ## LIGHTGBM
                            "seed_val": seed_val})
        
        print("AVG METRICS REGRESSION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_regre].mean(),4), round(df_val_cv[target_regre].mean(),4), round(df_val[target_regre].mean(),4)))
        print("Mean Preds:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(pred_train_cbtm),4), round(np.mean(pred_val_cbtm),4), round(np.mean(pred_val_out_cbtm),4)))
        print("Std Preds:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(pred_train_cbtm),4), round(np.std(pred_val_cbtm),4), round(np.std(pred_val_out_cbtm),4)))
        
      else:
        results_models.append({"rmse_train_cbtm": metric_reg_train_cbtm, "rmse_val_cbtm": metric_reg_val_cbtm, "rmse_val_out_cbtm": metric_reg_val_out_cbtm, ## CATBOOST
                            "rmse_train_lgbm": metric_reg_train_lgbm, "rmse_val_lgbm": metric_reg_val_lgbm, "rmse_val_out_lgbm": metric_reg_val_out_lgbm, ## LIGHTGBM
                            "seed_val": seed_val})

        print("AVG METRICS REGRESSION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_regre].mean(),4), round(df_val_cv[target_regre].mean(),4)))
        print("Mean Preds:\tTrain: {0},\tVal In: {1}".format(round(np.mean(pred_train_cbtm),4), round(np.mean(pred_val_cbtm),4)))
        print("Std Preds:\tTrain: {0},\tVal In: {1}".format(round(np.std(pred_train_cbtm),4), round(np.std(pred_val_cbtm),4)))
      
      list_models_cv.append({"cbtm_regre": cbt_reg_model, "lgbm_regre": cbt_reg_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": pred_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": pred_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": pred_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": pred_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  if df_val is not None:
    df_probs_val["pred_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "pred_cbtm" in c]].mean(axis=1)
    df_probs_val["pred_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "pred_lgbm" in c]].mean(axis=1)
  
  df_probs_test["pred_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "pred_cbtm" in c]].mean(axis=1)
  df_probs_test["pred_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "pred_lgbm" in c]].mean(axis=1)
  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

###### With ALL

In [0]:
results_renta_3, list_models_renta_3, df_probs_train_renta_3, df_probs_val_renta_3, df_probs_test_renta_3 = renta_regre_model_with_cv(df_train, df_test, target_regre, predictores, None)

Iteracion 0
	 REGRESIÓN
		 1. CATBOOST


KeyboardInterrupt: ignored

In [0]:
results_renta_3, list_models_renta_3, df_probs_train_renta_3, df_probs_val_renta_3, df_probs_test_renta_3 = renta_regre_model_with_cv(df_train, df_test, target_regre, predictores, None)

Iteracion 0
	 REGRESIÓN
		 1. CATBOOST
0:	learn: 70.6323288	test: 68.1838852	best: 68.1838852 (0)	total: 10.9ms	remaining: 10.9s
200:	learn: 67.0365837	test: 66.5086531	best: 66.5072984 (198)	total: 1.75s	remaining: 6.97s
400:	learn: 65.4214689	test: 66.2804175	best: 66.2799861 (399)	total: 3.4s	remaining: 5.08s
600:	learn: 64.0124861	test: 66.1598327	best: 66.1566030 (573)	total: 5.06s	remaining: 3.36s
800:	learn: 62.9469744	test: 66.0989939	best: 66.0988260 (797)	total: 6.7s	remaining: 1.66s
999:	learn: 62.0420351	test: 66.0736920	best: 66.0730259 (992)	total: 8.34s	remaining: 0us
bestTest = 66.07302586
bestIteration = 992
Shrink model to first 993 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's l2: 4605.06	valid_1's l2: 4453.55
[40]	training's l2: 4389.68	valid_1's l2: 4373.93
[60]	training's l2: 4228.02	valid_1's l2: 4348.06
[80]	training's l2: 4124.85	valid_1's l2: 4337.28
[100]	training's l2: 4042.7	valid_1's l2: 4329.61
[

In [0]:
pd.DataFrame(results_renta_3)

,rmse_train_cbtm,rmse_val_cbtm,rmse_val_out_cbtm,rmse_train_lgbm,rmse_val_lgbm,rmse_val_out_lgbm,seed_val
0,62.067973,66.073031,0,63.120145,65.757876,0,41
1,62.151660,64.519157,0,62.629153,64.112055,0,41
2,60.935919,74.497947,0,59.075205,73.867012,0,41
3,61.670082,70.510619,0,61.588932,69.879617,0,41
4,62.224102,63.282688,0,63.834563,63.346937,0,41


###### With Just Contacted

In [0]:
results_renta_4, list_models_renta_4, df_probs_train_renta_4, df_probs_val_renta_4, df_probs_test_renta_4 = renta_regre_model_with_cv(df_train.loc[df_train[target_adqui] == 1].reset_index(drop=True), df_test, target_regre, predictores, None)

Iteracion 0
	 REGRESIÓN
		 1. CATBOOST
0:	learn: 165.6326535	test: 164.9121976	best: 164.9121976 (0)	total: 42.6ms	remaining: 42.5s
200:	learn: 144.5103270	test: 154.2040869	best: 154.1819685 (193)	total: 6.89s	remaining: 27.4s
400:	learn: 139.5819079	test: 153.9507379	best: 153.8640302 (292)	total: 13.3s	remaining: 19.9s
600:	learn: 136.4130676	test: 154.0486134	best: 153.8640302 (292)	total: 19.8s	remaining: 13.1s
800:	learn: 134.5859888	test: 154.1745727	best: 153.8640302 (292)	total: 26.2s	remaining: 6.5s
999:	learn: 133.3539299	test: 154.2479791	best: 153.8640302 (292)	total: 32.4s	remaining: 0us
bestTest = 153.8640302
bestIteration = 292
Shrink model to first 293 iterations.
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's l2: 22622.2	valid_1's l2: 24327.5
[40]	training's l2: 20096.4	valid_1's l2: 23637.4
[60]	training's l2: 18826.3	valid_1's l2: 23346.4
[80]	training's l2: 18022	valid_1's l2: 23389.8
[100]	training's l2: 17356.3	valid

In [0]:
pd.DataFrame(results_renta_4)

,rmse_train_cbtm,rmse_val_cbtm,rmse_val_out_cbtm,rmse_train_lgbm,rmse_val_lgbm,rmse_val_out_lgbm,seed_val
0,141.993084,153.864032,0,136.337931,152.707743,0,41
1,140.430136,152.153236,0,124.064279,151.603005,0,41
2,132.284340,148.787830,0,124.973958,147.385737,0,41
3,132.778789,166.285008,0,131.397301,165.478383,0,41
4,132.558970,147.295191,0,130.328657,146.174203,0,41


In [0]:
df_probs_train_renta_4_complement = df_train.loc[df_train[target_adqui] != 1, :].copy().reset_index(drop=True)

In [0]:
for i, pair_m in enumerate(list_models_renta_4):
  print(i)
  df_probs_train_renta_4_complement["pred_cbtm_{}".format(i)] = pair_m["cbtm_regre"].predict(df_probs_train_renta_4_complement[predictores])
  df_probs_train_renta_4_complement["pred_lgbm_{}".format(i)] = pair_m["lgbm_regre"].predict(df_probs_train_renta_4_complement[predictores])

0
1
2
3
4


In [0]:
df_probs_train_renta_4_complement["pred_cbtm"] = df_probs_train_renta_4_complement[[c for c in df_probs_train_renta_4_complement.columns if "pred_cbtm" in c]].mean(axis=1)
df_probs_train_renta_4_complement["pred_lgbm"] = df_probs_train_renta_4_complement[[c for c in df_probs_train_renta_4_complement.columns if "pred_lgbm" in c]].mean(axis=1)

In [0]:
df_probs_train_renta_4_complement = df_probs_train_renta_4_complement[['prediction_id', 'margen', 'pred_cbtm', 'pred_lgbm']]

In [0]:
df_probs_train_renta_4_complement.head()

,prediction_id,margen,pred_cbtm,pred_lgbm
0,165747_201901,-5.0,3.912806,3.912806
1,58047_201901,-5.0,-3.028581,-3.028581
2,113221_201901,-5.0,98.321962,98.321962
3,142634_201901,-5.0,161.989300,161.989300
4,13204_201901,-5.0,77.155143,77.155143


In [0]:
df_probs_train_renta_4_final = pd.concat([df_probs_train_renta_4, df_probs_train_renta_4_complement], sort=False).reset_index(drop=True)

In [0]:
df_probs_train_renta_4_final.head()

,prediction_id,margen,pred_cbtm,pred_lgbm,FOLD
0,65751_201901,342.98999,266.003885,164.668865,FOLD_3
1,65634_201901,-3.62000,12.220110,13.120449,FOLD_3
2,86617_201901,521.98999,178.808543,185.954318,FOLD_3
3,116521_201901,40.50000,113.288662,105.749760,FOLD_3
4,178404_201901,-5.00000,47.518573,0.990993,FOLD_1


In [0]:
df_probs_train_renta_4_final["prediction_id"].nunique()

212665

In [0]:
df_probs_train_renta_4_final.shape

(212665, 5)

In [0]:
df_probs_train_renta_4_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212665 entries, 0 to 212664
Data columns (total 5 columns):
prediction_id    212665 non-null object
margen           212665 non-null float32
pred_cbtm        212665 non-null float64
pred_lgbm        212665 non-null float64
FOLD             30972 non-null object
dtypes: float32(1), float64(2), object(2)
memory usage: 7.3+ MB


### RESULTS

### Optimization

In [0]:
#df_probs_train_contact, df_probs_val_contact, df_probs_test_contact
df_probs_train_adqui.head()

,prediction_id,codtarget,prob_cbtm,prob_lgbm,FOLD
0,165747_201901,0.0,0.009868,0.010716,FOLD_2
1,65751_201901,1.0,0.279016,0.379954,FOLD_3
2,58047_201901,0.0,0.036175,0.040223,FOLD_4
3,113221_201901,0.0,0.156156,0.171142,FOLD_5
4,142634_201901,0.0,0.143672,0.124387,FOLD_3


In [0]:
#results_renta_1, list_models_renta_1, df_probs_train_renta_1, df_probs_val_renta_1, df_probs_test_renta_1
df_probs_train_renta_1.head()

,prediction_id,new_target,prob_cbtm,prob_lgbm,FOLD
0,165747_201901,0.0,0.004600,0.009550,FOLD_2
1,65751_201901,1.0,0.294350,0.237664,FOLD_5
2,58047_201901,0.0,0.010029,0.009274,FOLD_4
3,113221_201901,0.0,0.081018,0.106675,FOLD_5
4,142634_201901,0.0,0.038269,0.013356,FOLD_3


In [0]:
#results_renta_2, list_models_renta_2, df_probs_train_renta_2, df_probs_val_renta_2, df_probs_test_renta_2
df_probs_train_renta_2.shape

(30972, 5)

In [0]:
#results_renta_3, list_models_renta_3, df_probs_train_renta_3, df_probs_val_renta_3, df_probs_test_renta_3
df_probs_train_renta_3.head()

,prediction_id,margen,pred_cbtm,pred_lgbm,FOLD
0,165747_201901,-5.00000,-8.724386,-5.244561,FOLD_2
1,65751_201901,342.98999,33.909921,46.397335,FOLD_4
2,58047_201901,-5.00000,-9.149315,-5.854026,FOLD_5
3,113221_201901,-5.00000,17.787679,9.148081,FOLD_3
4,142634_201901,-5.00000,18.100429,24.941095,FOLD_4


In [0]:
#results_renta_4, list_models_renta_4, df_probs_train_renta_4, df_probs_val_renta_4, df_probs_test_renta_4
df_probs_train_renta_4.shape

(30972, 5)

In [0]:
#df_probs_val_renta_1[["prediction_id", "prob_avg_cbtm"]]
#df_probs_val_renta_1

## Catboost

In [0]:
df_val_opt_woc = df_probs_train_adqui[["prediction_id", "codtarget", "prob_cbtm"]].rename(columns={"prob_cbtm": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_train_renta_1[["prediction_id", "new_target", "prob_cbtm"]].rename(columns={"prob_cbtm": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_train_renta_3[["prediction_id", "margen", "pred_cbtm"]], how="left", on="prediction_id")\

In [0]:
df_val_opt_wic = df_probs_train_adqui[["prediction_id", "codtarget", "prob_cbtm"]].rename(columns={"prob_cbtm": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_train_renta_2_final[["prediction_id", "new_target", "prob_cbtm"]].rename(columns={"prob_cbtm": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_train_renta_4_final[["prediction_id", "margen", "pred_cbtm"]], how="left", on="prediction_id")\

In [0]:
df_test_opt_woc = df_probs_test_adqui[["prediction_id", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_test_renta_1[["prediction_id", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_test_renta_3[["prediction_id", "pred_avg_cbtm"]].rename(columns={"pred_avg_cbtm": "pred_cbtm"}), how="left", on="prediction_id")\

In [0]:
df_test_opt_wic = df_probs_test_adqui[["prediction_id", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_test_renta_2[["prediction_id", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_test_renta_4[["prediction_id", "pred_avg_cbtm"]].rename(columns={"pred_avg_cbtm": "pred_cbtm"}), how="left", on="prediction_id")\

## HIBRIDO

In [0]:
df_probs_train_renta_3 = pd.read_csv("[01] Intermediate/df_probs_train_renta_3.csv")
df_probs_test_renta_3 = pd.read_csv("[01] Intermediate/df_probs_test_renta_3.csv")
df_probs_train_renta_4_final = pd.read_csv("[01] Intermediate/df_probs_train_renta_4_final.csv")
df_probs_test_renta_4 = pd.read_csv("[01] Intermediate/df_probs_test_renta_4.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_probs_train_renta_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248734 entries, 0 to 248733
Data columns (total 5 columns):
prediction_id    248734 non-null object
margen           248734 non-null float64
pred_cbtm        248734 non-null float64
pred_lgbm        248734 non-null float64
FOLD             248734 non-null object
dtypes: float64(3), object(2)
memory usage: 9.5+ MB


In [0]:
df_probs_test_renta_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142616 entries, 0 to 142615
Data columns (total 13 columns):
prediction_id    142616 non-null object
pred_cbtm_0      142616 non-null float64
pred_lgbm_0      142616 non-null float64
pred_cbtm_1      142616 non-null float64
pred_lgbm_1      142616 non-null float64
pred_cbtm_2      142616 non-null float64
pred_lgbm_2      142616 non-null float64
pred_cbtm_3      142616 non-null float64
pred_lgbm_3      142616 non-null float64
pred_cbtm_4      142616 non-null float64
pred_lgbm_4      142616 non-null float64
pred_avg_cbtm    142616 non-null float64
pred_avg_lgbm    142616 non-null float64
dtypes: float64(12), object(1)
memory usage: 14.1+ MB


In [0]:
df_probs_train_renta_4_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248734 entries, 0 to 248733
Data columns (total 5 columns):
prediction_id    248734 non-null object
margen           248734 non-null float64
pred_cbtm        248734 non-null float64
pred_lgbm        248734 non-null float64
FOLD             30972 non-null object
dtypes: float64(3), object(2)
memory usage: 9.5+ MB


In [0]:
df_probs_test_renta_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142616 entries, 0 to 142615
Data columns (total 13 columns):
prediction_id    142616 non-null object
pred_cbtm_0      142616 non-null float64
pred_lgbm_0      142616 non-null float64
pred_cbtm_1      142616 non-null float64
pred_lgbm_1      142616 non-null float64
pred_cbtm_2      142616 non-null float64
pred_lgbm_2      142616 non-null float64
pred_cbtm_3      142616 non-null float64
pred_lgbm_3      142616 non-null float64
pred_cbtm_4      142616 non-null float64
pred_lgbm_4      142616 non-null float64
pred_avg_cbtm    142616 non-null float64
pred_avg_lgbm    142616 non-null float64
dtypes: float64(12), object(1)
memory usage: 14.1+ MB


In [0]:
df_probs_train_adqui["prob"] = df_probs_train_adqui[["prob_cbtm", "prob_lgbm"]].mean(axis=1)
df_probs_train_renta_2_final["prob"] = df_probs_train_renta_2_final[["prob_cbtm", "prob_lgbm"]].mean(axis=1)
df_probs_train_renta_4_final["pred"] = df_probs_train_renta_4_final[["pred_cbtm", "pred_lgbm"]].mean(axis=1)
df_probs_train_renta_1["prob"] = df_probs_train_renta_1[["prob_cbtm", "prob_lgbm"]].mean(axis=1)
df_probs_train_renta_3["pred"] = df_probs_train_renta_3[["pred_cbtm", "pred_lgbm"]].mean(axis=1)

In [0]:
df_probs_test_adqui["prob"] = df_probs_test_adqui[["prob_avg_cbtm", "prob_avg_lgbm"]].mean(axis=1)
df_probs_test_renta_2["prob"] = df_probs_test_renta_2[["prob_avg_cbtm", "prob_avg_lgbm"]].mean(axis=1)
df_probs_test_renta_4["pred"] = df_probs_test_renta_4[["pred_avg_cbtm", "pred_avg_lgbm"]].mean(axis=1)
df_probs_test_renta_1["prob"] = df_probs_test_renta_1[["prob_avg_cbtm", "prob_avg_lgbm"]].mean(axis=1)
df_probs_test_renta_3["pred"] = df_probs_test_renta_3[["pred_avg_cbtm", "pred_avg_lgbm"]].mean(axis=1)

In [0]:
df_val_opt_woc_h = df_probs_train_adqui[["prediction_id", "codtarget", "prob"]].rename(columns={"prob": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_train_renta_1[["prediction_id", "new_target", "prob"]].rename(columns={"prob": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_train_renta_3[["prediction_id", "margen", "pred"]].rename(columns={"pred": "pred_cbtm"}), how="left", on="prediction_id")\

In [0]:
df_val_opt_wic_h = df_probs_train_adqui[["prediction_id", "codtarget", "prob"]].rename(columns={"prob": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_train_renta_2_final[["prediction_id", "new_target", "prob"]].rename(columns={"prob": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_train_renta_4_final[["prediction_id", "margen", "pred"]].rename(columns={"pred": "pred_cbtm"}), how="left", on="prediction_id")\

In [0]:
df_test_opt_woc_h = df_probs_test_adqui[["prediction_id", "prob"]].rename(columns={"prob": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_test_renta_1[["prediction_id", "prob"]].rename(columns={"prob": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_test_renta_3[["prediction_id", "pred"]].rename(columns={"pred": "pred_cbtm"}), how="left", on="prediction_id")\

In [0]:
df_test_opt_wic_h = df_probs_test_adqui[["prediction_id", "prob"]].rename(columns={"prob": "adqui_prob_cbtm"})\
                                                          .merge(df_probs_test_renta_2[["prediction_id", "prob"]].rename(columns={"prob": "renta_prob_cbtm"}), how="left", on="prediction_id")\
                                                          .merge(df_probs_test_renta_4[["prediction_id", "pred"]].rename(columns={"pred": "pred_cbtm"}), how="left", on="prediction_id")\

In [0]:
df_val_opt_wic_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248734 entries, 0 to 248733
Data columns (total 7 columns):
prediction_id      248734 non-null object
codtarget          248734 non-null float32
adqui_prob_cbtm    248734 non-null float64
new_target         248734 non-null float32
renta_prob_cbtm    248734 non-null float64
margen             248734 non-null float64
pred_cbtm          248734 non-null float64
dtypes: float32(2), float64(4), object(1)
memory usage: 13.3+ MB


In [0]:
df_test_opt_wic_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142616 entries, 0 to 142615
Data columns (total 4 columns):
prediction_id      142616 non-null object
adqui_prob_cbtm    142616 non-null float64
renta_prob_cbtm    142616 non-null float64
pred_cbtm          142616 non-null float64
dtypes: float64(3), object(1)
memory usage: 5.4+ MB


In [0]:
df_val_opt_woc.info()

In [0]:
df_val_opt_wic.info()

In [0]:
df_val_opt_woc.head()

In [0]:
df_val_opt_wic.head()

In [0]:
def clasificar_old(res, c, prob_col, pred_col):
    return ((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * c[5] > c[6]

def cost_old(res, coefs, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_old(res, coefs, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
# 7 parameters to optimize
optimization_val_cbtm_old = differential_evolution(lambda x: cost_old(df_val_opt_woc_h, x,
                                                                      "renta_prob_cbtm",
                                                                      "pred_cbtm",
                                                                      target_class, target_regre),
                                                   [(0, 1),(0, 20),(0, 1),(0, 1),(200, 300),(0, 1),(0, 2)])
optimization_val_cbtm_old

     fun: -0.7171107210849191
 message: 'Optimization terminated successfully.'
    nfev: 2108
     nit: 19
 success: True
       x: array([4.92059141e-02, 1.00456039e+01, 7.99363962e-01, 3.00101880e-01,
       2.48179829e+02, 2.36435848e-01, 2.56177657e-01])

In [0]:
clasificar_old(df_val_opt_woc, optimization_val_cbtm_old["x"], "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar_old(df_test_opt_woc, optimization_val_cbtm_old["x"], "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
# 8 parameters to optimize
def clasificar(res, c, prob_col_contact, prob_col, pred_col):
    return (res[prob_col_contact] > c[7]) * (((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * c[5]) > c[6]

def cost(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new = differential_evolution(lambda x: cost(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(0, 200),(0, 1),(0, 2),(0, 1)])
optimization_val_cbtm_new

     fun: -0.7167252506335423
 message: 'Optimization terminated successfully.'
    nfev: 2049
     nit: 16
 success: True
       x: array([8.85386334e-01, 7.47379203e+01, 5.54275592e-01, 4.18715631e-01,
       3.43273948e+01, 6.33669202e-01, 1.23307860e-01, 6.93611109e-02])

In [0]:
clasificar(df_val_opt_wic, optimization_val_cbtm_new["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar(df_test_opt_wic, optimization_val_cbtm_new["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
optimization_val_cbtm_new_woc = differential_evolution(lambda x: cost(df_val_opt_woc,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(100, 200),(0, 1),(0, 2),(0, 1)])
optimization_val_cbtm_new_woc

In [0]:
clasificar(df_val_opt_woc, optimization_val_cbtm_new_woc["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar(df_test_opt_woc, optimization_val_cbtm_new_woc["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
def clasificar_v2(res, c, prob_col_contact, prob_col, pred_col):
    return res[prob_col_contact] * (((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * c[5]) > c[6]

def cost_v2(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v2(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v2 = differential_evolution(lambda x: cost_v2(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(0, 200),(0, 1),(0, 2)])
optimization_val_cbtm_new_v2

     fun: -0.7253838980599233
 message: 'Optimization terminated successfully.'
    nfev: 1268
     nit: 11
 success: True
       x: array([7.60455285e-01, 1.68759452e+01, 3.91021229e-01, 4.74378235e-01,
       6.03403025e+01, 5.74716929e-01, 5.11098755e-02])

In [0]:
clasificar_v2(df_val_opt_wic, optimization_val_cbtm_new_v2["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar_v2(df_val_opt_wic, optimization_val_cbtm_new_v2["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
optimization_val_cbtm_new_v2_woc = differential_evolution(lambda x: cost_v2(df_val_opt_woc_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(0, 200),(0, 1),(0, 2)])
optimization_val_cbtm_new_v2_woc

     fun: -0.7209835161371787
 message: 'Optimization terminated successfully.'
    nfev: 2423
     nit: 22
 success: True
       x: array([7.39913715e-02, 4.17791723e+00, 7.24482521e-01, 4.79650923e-02,
       8.39082753e+01, 4.52298739e-01, 2.98898701e-02])

In [0]:
# Optimize 
def clasificar_v3(res, c, prob_col_contact, prob_col, pred_col):
    return res[prob_col_contact] * (((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * (1 - c[2])) > c[5]

def cost_v3(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v3(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v3 = differential_evolution(lambda x: cost_v3(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(0, 200),(0, 2)])
optimization_val_cbtm_new_v3

     fun: -0.7245445530776942
 message: 'Optimization terminated successfully.'
    nfev: 1357
     nit: 14
 success: True
       x: array([6.66422253e-01, 1.84080754e+01, 3.74223558e-01, 4.32306214e-01,
       8.23478764e+01, 4.22840724e-02])

In [0]:
clasificar_v3(df_val_opt_wic, optimization_val_cbtm_new_v3["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar_v3(df_test_opt_wic, optimization_val_cbtm_new_v3["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
optimization_val_cbtm_new_v3_woc = differential_evolution(lambda x: cost_v3(df_val_opt_woc_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 1),(0, 200),(0, 2)])
optimization_val_cbtm_new_v3_woc

     fun: -0.7207698693940029
 message: 'Optimization terminated successfully.'
    nfev: 1627
     nit: 17
 success: True
       x: array([7.02992069e-02, 4.39817453e+00, 8.71656329e-01, 6.11529565e-02,
       1.29526302e+02, 3.72394836e-02])

In [0]:
clasificar_v3(df_val_opt_woc, optimization_val_cbtm_new_v3_woc["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar_v3(df_test_opt_woc, optimization_val_cbtm_new_v3_woc["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
# Optimize 
def clasificar_v4(res, c, prob_col_contact, prob_col, pred_col):
    return res[prob_col_contact] * (((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[0]) & (res[pred_col] > c[1])) * (1 - c[2])) > c[3]

def cost_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4

     fun: -0.7251532375796401
 message: 'Optimization terminated successfully.'
    nfev: 905
     nit: 14
 success: True
       x: array([ 0.384313  , 63.28076968,  0.52367819,  0.06329589])

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_woc_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4

     fun: -0.7212512380558953
 message: 'Optimization terminated successfully.'
    nfev: 1025
     nit: 16
 success: True
       x: array([0.07949983, 2.44500513, 0.64701103, 0.04234438])

In [0]:
##### 1st BEST OPTIMIZATION MODEL
# Optimize 
def clasificar_v4(res, c, prob_col_contact, prob_col, pred_col):
    return (((res[prob_col_contact] * res[prob_col] > c[0]) | (res[prob_col_contact] * res[pred_col] > c[1])) * c[2] + \
            ((res[prob_col_contact] * res[prob_col] > c[0]) & (res[prob_col_contact] * res[pred_col] > c[1])) * (1 - c[2])) > c[3]

def cost_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_woc_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4

     fun: -0.7222160151212436
 message: 'Optimization terminated successfully.'
    nfev: 1325
     nit: 21
 success: True
       x: array([0.01251694, 0.44103609, 0.34516944, 0.21944948])

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4

     fun: -0.7282122714424242
 message: 'Optimization terminated successfully.'
    nfev: 725
     nit: 11
 success: True
       x: array([0.52388564, 5.87831595, 0.86835394, 0.28305198])

In [0]:
##### BEST OPTIMIZATION MODEL
# Optimize 
def clasificar_v4(res, c, prob_col_contact, prob_col, pred_col):
    return (((res[prob_col_contact] * res[prob_col] > c[0]) | (res[prob_col_contact] * res[prob_col] * res[pred_col] > c[1])) * c[2] + \
            ((res[prob_col_contact] * res[prob_col] > c[0]) & (res[prob_col_contact] * res[prob_col] * res[pred_col] > c[1])) * (1 - c[2])) > c[3]

def cost_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v4(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_woc_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4

     fun: -0.7140524139670227
     jac: array([0., 0., 0., 0.])
 message: 'Optimization terminated successfully.'
    nfev: 1140
     nit: 17
 success: True
       x: array([4.57484234e-03, 5.40307357e+01, 7.31648186e-01, 3.77720941e-01])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 20),(0, 1),(0, 1)])
optimization_val_cbtm_new_v4_h

     fun: -0.7318257462623142
 message: 'Optimization terminated successfully.'
    nfev: 485
     nit: 7
 success: True
       x: array([0.33449047, 3.42094727, 0.17637854, 0.07966141])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4_h

     fun: -0.7318816980750626
 message: 'Optimization terminated successfully.'
    nfev: 1265
     nit: 20
 success: True
       x: array([0.91703377, 3.46401685, 0.99854517, 0.4137564 ])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 20),(0, 1),(0, 1)])
optimization_val_cbtm_new_v4_h

     fun: -0.7463694214820862
 message: 'Optimization terminated successfully.'
    nfev: 425
     nit: 6
 success: True
       x: array([0.10707743, 6.11174829, 0.78600495, 0.57716555])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 1)])
optimization_val_cbtm_new_v4_h

     fun: -0.7502391338348389
 message: 'Optimization terminated successfully.'
    nfev: 1025
     nit: 16
 success: True
       x: array([0.43383366, 3.76116376, 0.84782059, 0.10868523])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 0.04),(0, 20),(0, 1),(0, 1)])
optimization_val_cbtm_new_v4_h

     fun: -0.7500177621841431
 message: 'Optimization terminated successfully.'
    nfev: 425
     nit: 6
 success: True
       x: array([0.00446437, 3.73240934, 0.02310338, 0.52764056])

In [0]:
optimization_val_cbtm_new_v4_h = differential_evolution(lambda x: cost_v4(df_val_opt_wic_h,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 1)])
optimization_val_cbtm_new_v4_h

     fun: -0.7502955794334412
 message: 'Optimization terminated successfully.'
    nfev: 1265
     nit: 20
 success: True
       x: array([0.80773249, 3.75354976, 0.32271108, 0.0307388 ])

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_wic,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0.4, 0.45),(50, 70),(0.4, 0.5),(0, 0.1)])
optimization_val_cbtm_new_v4

     fun: -0.3347477912902832
 message: 'Optimization terminated successfully.'
    nfev: 725
     nit: 11
 success: True
       x: array([4.00470007e-01, 5.00426651e+01, 4.39207321e-01, 1.49516098e-03])

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_wic,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0,1),(0, 1)])
optimization_val_cbtm_new_v4

In [0]:
[(0, 0.5),(0, 10),(0.5,1),(0, 1)])

In [0]:
optimization_val_cbtm_new_v4 = differential_evolution(lambda x: cost_v4(df_val_opt_woc,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0,1),(0, 1)])
optimization_val_cbtm_new_v4

In [0]:
df_val_opt_wic["class_predict"] = clasificar_v4(df_val_opt_wic, optimization_val_cbtm_new_v4["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int)

In [0]:
del df_val_opt_wic["class_adqui"]
del df_val_opt_wic["class_offer"]

In [0]:
df_val_opt_wic.head()

,prediction_id,codtarget,adqui_prob_cbtm,new_target,renta_prob_cbtm,margen,pred_cbtm
0,165747_201901,0.0,0.009868,0.0,0.543477,-5.00000,3.912806
1,65751_201901,1.0,0.279016,1.0,0.715803,342.98999,266.003885
2,58047_201901,0.0,0.036175,0.0,0.204742,-5.00000,-3.028581
3,113221_201901,0.0,0.156156,0.0,0.755409,-5.00000,98.321962
4,142634_201901,0.0,0.143672,0.0,0.530932,-5.00000,161.989300


In [0]:
df_val_opt_wic.loc[(df_val_opt_wic["class_predict"] == 0) & (df_val_opt_wic["new_target"] == 1)]

In [0]:
optimization_val_cbtm_new_v4_woc = differential_evolution(lambda x: cost_v4(df_val_opt_woc,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2)])
optimization_val_cbtm_new_v4_woc

In [0]:
def offer_product(res, c, prob_col_contact):
    return (res[prob_col_contact]) > c

def clasificar_v5(res, c, prob_col_contact, prob_col, pred_col):
    return (((offer_product(res, c[4], prob_col_contact) * res[prob_col]) > c[0]) | ((offer_product(res, c[4], prob_col_contact) * res[pred_col]) > c[1])) * c[2] + \
            (((offer_product(res, c[4], prob_col_contact) * res[prob_col]) > c[0]) & ((offer_product(res, c[4], prob_col_contact) * res[pred_col]) > c[1])) * (1 - c[2]) > c[3]

def cost_v5(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    return -((clasificar_v5(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / (res[target_class] * res[target_regre]).sum())

In [0]:
optimization_val_cbtm_new_v5 = differential_evolution(lambda x: cost_v5(df_val_opt_wic,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(0, 100),(0, 1),(0, 2),(0, 1)])
optimization_val_cbtm_new_v5

In [0]:
clasificar_v5(df_val_opt_wic, optimization_val_cbtm_new_v3["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
def clasificar_v6(res, c, prob_col_contact, prob_col, pred_col):
    return res[prob_col_contact] * (((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * c[5]) > c[6]

def cost_v6(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    res["class_opt"] = clasificar_v6(res, coefs, prob_col_contact, prob_col_renta, pred_col)
    
    res_success = res.loc[(res["class_opt"] == 1) & (res[target_class] == 1)].reset_index(drop=True)
    rentability_sum = (res_success[target_regre]).sum()

    res_fail = res.loc[(res["class_opt"] == 1) & (res[target_class] == 0)].reset_index(drop=True)
    rentability_loss = (res_fail[target_regre]).sum()

    rentability_perfect = (res[target_class] * res[target_regre]).sum()


    return -((rentability_sum + rentability_loss)/ rentability_perfect)

In [0]:
optimization_val_cbtm_new_v6 = differential_evolution(lambda x: cost_v6(df_val_opt_wic,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 1),(-100, 500),(0, 1),(0, 2)])
optimization_val_cbtm_new_v6

In [0]:
df_val_opt_wic.head()

In [0]:
df_val_opt_wic["class_opt"].value_counts()

In [0]:
clasificar_v6(df_val_opt_wic, optimization_val_cbtm_new_v6["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
clasificar_v6(df_test_opt_wic, optimization_val_cbtm_new_v6["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
def clasificar_adqui(res, prob_col_contact, c):
    return (res[prob_col_contact] > c)

def clasificar_v7(res, c, prob_col_contact, prob_col, pred_col):
    res["class_adqui"] = clasificar_adqui(res, prob_col_contact, c[7])
    res["class_offer"] = ((res[prob_col] > c[0]) | (res[pred_col] > c[1])) * c[2] + ((res[prob_col] > c[3]) & (res[pred_col] > c[4])) * c[5] > c[6]
    
    res.loc[res["class_adqui"] == 0, "class_offer"] = 0
    return res["class_offer"]

def cost_v7(res, coefs, prob_col_contact, prob_col_renta, pred_col, target_class, target_regre):
    rentability_perfect = (res[target_class] * res[target_regre]).sum()
    return -((clasificar_v7(res, coefs, prob_col_contact, prob_col_renta, pred_col) * res[target_regre]).sum() / rentability_perfect)

In [0]:
optimization_val_cbtm_new_v7 = differential_evolution(lambda x: cost_v7(df_val_opt_wic,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 1),(-100, 500),(0, 1),(0, 2), (0, 0.5)])
optimization_val_cbtm_new_v7

In [0]:
clasificar_v7(df_val_opt_wic, optimization_val_cbtm_new_v6["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int).value_counts(normalize=True, dropna=False)

In [0]:
df_val_opt_wic.loc[df_val_opt_wic["adqui_prob_cbtm"] > 0.0692059702]

In [0]:
df_val_opt_wic["adqui_prob_cbtm"].describe()

In [0]:
df_val_opt_wic.head()

In [0]:
optimization_val_cbtm_new_v7_woc = differential_evolution(lambda x: cost_v7(df_val_opt_woc,
                                                                  x,
                                                                  "adqui_prob_cbtm",
                                                                  "renta_prob_cbtm",
                                                                  "pred_cbtm",
                                                                  target_class, target_regre),
                                                    [(0, 1),(-100, 500),(0, 1),(0, 1),(-100, 500),(0, 1),(0, 2), (0, 1)])
optimization_val_cbtm_new_v7_woc

In [0]:
df_val_opt_wic.head()

In [0]:
pd.pivot_table(df_val_opt_wic, index="codtarget", columns="new_target", values="prediction_id", aggfunc="count")

In [0]:
(df_val_opt_wic["margen"] * df_val_opt_wic["new_target"]).sum()

In [0]:
(df_val_opt_wic.loc[df_val_opt_wic["codtarget"] == 1, "margen"].reset_index(drop=True) * df_val_opt_wic.loc[df_val_opt_wic["codtarget"] == 1, "new_target"].reset_index(drop=True)).sum()

In [0]:
df_test_opt_wic["class"] = clasificar_v4(df_test_opt_wic, optimization_val_cbtm_new_v4["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int)

In [0]:
df_test_opt_wic["class"].value_counts(normalize=True)

0    0.560797
1    0.439203
Name: class, dtype: float64

In [0]:
df_test_opt_wic_h["class"] = clasificar_v4(df_test_opt_wic_h, optimization_val_cbtm_new_v4_h["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int)

In [0]:
df_test_opt_wic_h.shape

(142616, 5)

In [0]:
complement_test = pd.DataFrame({"prediction_id": list_leakage_id, "class": 0})

In [0]:
submit_milagroso = pd.concat([df_test_opt_wic_h[["prediction_id", "class"]], complement_test]).reset_index(drop=True)

In [0]:
submit_milagroso["class"].value_counts(normalize=True, dropna=False)

0    0.776439
1    0.223561
Name: class, dtype: float64

In [0]:
submit_milagroso[["prediction_id", "class"]].to_csv('baseline.csv', index=False)

In [0]:
df_test_opt_wic_h.shape

(142616, 5)

In [0]:
df_base_all.loc[df_base_all["codmes"].isin([201905, 201906, 201907])].shape

(178685, 16)

In [0]:
df_test_opt_wic_h.head()

,prediction_id,adqui_prob_cbtm,renta_prob_cbtm,pred_cbtm,class
0,211794_201905,0.029452,0.638486,78.400945,0
1,140704_201905,0.165230,0.703646,50.620426,1
2,80734_201905,0.025866,0.650901,52.621511,0
3,20181_201905,0.025130,0.622966,232.224685,1
4,69634_201905,0.036385,0.466578,37.145181,0


In [0]:
df_test_opt_wic_h["class"].value_counts(normalize=True)

0    0.719898
1    0.280102
Name: class, dtype: float64

In [0]:
df_test_opt_wic_h["class"] = clasificar_v4(df_test_opt_wic_h, optimization_val_cbtm_new_v4_h["x"], "adqui_prob_cbtm", "renta_prob_cbtm", "pred_cbtm").astype(int)

In [0]:
df_test_opt_wic_h["class"].value_counts(normalize=True)

0    0.589479
1    0.410521
Name: class, dtype: float64

In [0]:
df_test_opt_wic_h["class"].value_counts(normalize=True)

0    0.572958
1    0.427042
Name: class, dtype: float64

In [0]:
df_test_opt_wic_h[["prediction_id", "class"]].to_csv('submission_18.csv', index=False)

In [0]:
submission_20 = pd.read_csv("submission_20.csv")

In [0]:
submission_20["class"].value_counts(normalize=True)

0    0.647922
1    0.352078
Name: class, dtype: float64

Pessimistic: -0.36700522899627686

Train Class CBTM: 0.722051739692688
Val Class CBTM: 0.7226264476776123
Train Class LGBM: 0.7186115980148315
Val Class LGBM: 0.7239195704460144

Train Old Class CBTM: 0.7212346792221069
Val Old Class CBTM: 0.7263498306274414
Train Old Class LGBM: 0.719330370426178
Val Old Class LGBM: 0.7233311533927917

In [0]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models_1[0]["cbtm_class"].get_feature_importance()})
for i in range(1, 5):
    print(i)
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models_1[i]["cbtm_class"].get_feature_importance()}), how="left", on="name")
    df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)

df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(20)